# Análisis y Visualizaciones — Notebook (estilo integrado)

**Autor:** Abraham Rey Sanchez Aamador  
**Asignatura:** Modelos de Optimización  
**Fecha:** 2025-11-12

Este notebook genera y guarda todas las figuras en `figures/` en formato PNG (alta resolución).

## 1) Dependencias y configuración
Se recomienda ejecutar esta celda al inicio para preparar el entorno.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import scipy.optimize as opt
from datetime import datetime

# Configuración global de matplotlib (tipografía y tamaño)
plt.rcParams.update({
    'font.size': 11,
    'font.family': 'serif',
    'figure.dpi': 150,
})

# Crear carpeta para figuras
FIG_DIR = 'figures'
os.makedirs(FIG_DIR, exist_ok=True)
print('Directorio de figuras:', os.path.abspath(FIG_DIR))

Directorio de figuras: d:\Abraham\Escuela\3ero\MO\TE2\src\figures


## 2) Definición de la función, gradiente y Hessiano

In [2]:
def f(xy):
    x, y = xy
    r2 = x**2 + y**2 + 1
    return -np.cos(np.log(r2))

def grad_f(xy):
    x, y = xy
    r2 = x**2 + y**2 + 1
    deriv = 2 * np.sin(np.log(r2)) / r2
    return np.array([deriv * x, deriv * y])

def hess_f(xy):
    x, y = xy
    r2 = x**2 + y**2 + 1
    log_r = np.log(r2)
    sin_log = np.sin(log_r)
    cos_log = np.cos(log_r)
    term1 = (2*sin_log)/r2 - (4*x**2*sin_log)/r2**2 + (4*x**2*cos_log)/r2**2
    term2 = - (4*x*y*sin_log)/r2**2 + (4*x*y*cos_log)/r2**2
    H = np.array([[term1, term2],[term2, term1]])
    return H

## 3) Utilidades de plotting
Funciones auxiliares para generar y guardar figuras de forma consistente.

In [3]:
def save_fig(fig, name, dpi=300, bbox_inches='tight'):
    path = os.path.join(FIG_DIR, name)
    fig.savefig(path, dpi=dpi, bbox_inches=bbox_inches)
    print('Guardado ->', path)

def make_mesh(xmin=-5, xmax=5, ymin=-5, ymax=5, n=300):
    x = np.linspace(xmin, xmax, n)
    y = np.linspace(ymin, ymax, n)
    X, Y = np.meshgrid(x, y)
    Z = f([X, Y])
    return x, y, X, Y, Z

## 4) Superficies y curvas de nivel
Generar 3D y curvas de nivel; guardar en PNG.

In [4]:
x, y, X, Y, Z = make_mesh(n=200)
x_s, y_s, X_s, Y_s, Z_s = make_mesh(xmin=-2, xmax=2, ymin=-2, ymax=2, n=200)
from mpl_toolkits.mplot3d import Axes3D

# Superficie completa
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=cm.viridis, linewidth=0, antialiased=True)
ax.set_title('Superficie 3D de f(x,y)')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x,y)')
fig.colorbar(surf, ax=ax, shrink=0.6)
save_fig(fig, 'superficie_completa.png')
plt.close(fig)

# Superficie cerca del origen
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X_s, Y_s, Z_s, cmap=cm.viridis, linewidth=0, antialiased=True)
ax.set_title('Superficie cerca del origen')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x,y)')
fig.colorbar(surf, ax=ax, shrink=0.6)
save_fig(fig, 'superficie_origen.png')
plt.close(fig)

Guardado -> figures\superficie_completa.png
Guardado -> figures\superficie_origen.png


## 5) Curvas de nivel y mapas de calor

In [5]:
# Curvas de nivel
fig, ax = plt.subplots(1,1, figsize=(7,6))
CS = ax.contour(X, Y, Z, levels=30)
ax.clabel(CS, inline=1, fontsize=8)
ax.set_title('Curvas de nivel (lineal)')
ax.set_aspect('equal')
save_fig(fig, 'curvas_nivel_lineal.png')
plt.close(fig)

# Curvas de nivel log10
Z_log = np.log10(np.abs(Z) + 1e-12) * np.sign(Z)
fig, ax = plt.subplots(1,1, figsize=(7,6))
CS = ax.contour(X, Y, Z_log, levels=30)
ax.clabel(CS, inline=1, fontsize=8)
ax.set_title('Curvas de nivel (log10)')
ax.set_aspect('equal')
save_fig(fig, 'curvas_nivel_log10.png')
plt.close(fig)

# Mapas de calor
fig, axs = plt.subplots(1,2, figsize=(14,6))
im1 = axs[0].imshow(Z, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='viridis')
axs[0].set_title('Mapa de calor (lineal)')
fig.colorbar(im1, ax=axs[0])
im2 = axs[1].imshow(Z_log, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='RdBu_r')
axs[1].set_title('Mapa de calor (log10)')
fig.colorbar(im2, ax=axs[1])
save_fig(fig, 'mapas_calor.png')
plt.close(fig)

Guardado -> figures\curvas_nivel_lineal.png
Guardado -> figures\curvas_nivel_log10.png
Guardado -> figures\mapas_calor.png


## 6) Cortes 1D

In [6]:
fig, axs = plt.subplots(1,2, figsize=(14,5))
y_values = [0, -2, 2]
for yv in y_values:
    Z_cut = f([x, yv*np.ones_like(x)])
    axs[0].plot(x, Z_cut, label=f'y={yv}')
axs[0].set_title('Cortes 1D (variando x)')
axs[0].legend()
x_values = [0, -2, 2]
for xv in x_values:
    Z_cut = f([xv*np.ones_like(y), y])
    axs[1].plot(y, Z_cut, label=f'x={xv}')
axs[1].set_title('Cortes 1D (variando y)')
axs[1].legend()
save_fig(fig, 'cortes_1d.png')
plt.close(fig)

Guardado -> figures\cortes_1d.png


## 7) Métodos de optimización y comparación de trayectorias

In [7]:
def maximo_descenso(x0, max_iter=1000, tol=1e-6):
    x = x0.copy()
    trajectory = [x.copy()]
    for i in range(max_iter):
        g = grad_f(x)
        if np.linalg.norm(g) < tol:
            break
        
        # Búsqueda lineal adaptativa con tamaño de paso inicial variable
        alpha = min(1.0, 1.0 / (np.linalg.norm(g) + 1e-8))
        for _ in range(50):  # Aumentado a 50 iteraciones de backtracking
            x_new = x - alpha * g
            if f(x_new) < f(x) - 1e-4 * alpha * np.linalg.norm(g)**2:
                break
            alpha *= 0.7  # Factor de reducción más conservador
        x = x_new
        trajectory.append(x.copy())
    return np.array(trajectory)

def metodo_newton(x0, max_iter=100, tol=1e-8):
    x = x0.copy()
    trajectory = [x.copy()]
    for i in range(max_iter):
        g = grad_f(x)
        H = hess_f(x)
        if np.linalg.norm(g) < tol:
            break
        try:
            H_reg = H + 1e-8 * np.eye(2)
            p = -np.linalg.solve(H_reg, g)
        except Exception:
            p = -g
        alpha = 1.0
        for _ in range(20):
            x_new = x + alpha * p
            if f(x_new) < f(x):
                break
            alpha *= 0.5
        x = x_new
        trajectory.append(x.copy())
    return np.array(trajectory)

puntos_iniciales = [np.array([4.0,4.0]), np.array([-3.0,2.0]), np.array([1.0,-4.0]), np.array([-2.0,-3.0])]
fig, axes = plt.subplots(2,2, figsize=(12,12))
axes = axes.flatten()
for idx, x0 in enumerate(puntos_iniciales):
    ax = axes[idx]
    CS = ax.contour(X, Y, Z, levels=15, alpha=0.5)
    traj_md = maximo_descenso(x0)
    ax.plot(traj_md[:,0], traj_md[:,1], 'o-', label='Máx. Descenso', linewidth=2, markersize=4)
    traj_newton = metodo_newton(x0)
    ax.plot(traj_newton[:,0], traj_newton[:,1], 's-', label='Newton', linewidth=2, markersize=4)
    ax.plot(x0[0], x0[1], 'k*', markersize=12, label='Inicio')
    ax.set_title(f'Punto inicial: ({x0[0]}, {x0[1]})')
    ax.set_aspect('equal')
    ax.legend()
save_fig(fig, 'comparacion_trayectorias.png')
plt.close(fig)

Guardado -> figures\comparacion_trayectorias.png


## 8) Análisis de convergencia

In [8]:
x0 = np.array([4.0,4.0])
traj_md = maximo_descenso(x0, max_iter=200)
traj_newton = metodo_newton(x0, max_iter=100)
f_md = [f(x) for x in traj_md]
f_newton = [f(x) for x in traj_newton]
grad_md = [np.linalg.norm(grad_f(x)) for x in traj_md]
grad_newton = [np.linalg.norm(grad_f(x)) for x in traj_newton]

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,6))
ax1.semilogy(range(len(f_md)), np.abs(np.array(f_md) - f(traj_md[-1])), 'o-', label='Máx. Descenso')
ax1.semilogy(range(len(f_newton)), np.abs(np.array(f_newton) - f(traj_newton[-1])), 's-', label='Newton')
ax1.set_xlabel('Iteración')
ax1.set_ylabel('|f(x_k) - f(x*)|')
ax1.set_title('Convergencia del valor de la función')
ax1.legend()
ax1.grid(True)
ax2.semilogy(range(len(grad_md)), grad_md, 'o-', label='Máx. Descenso')
ax2.semilogy(range(len(grad_newton)), grad_newton, 's-', label='Newton')
ax2.set_xlabel('Iteración')
ax2.set_ylabel('||∇f(x_k)||')
ax2.set_title('Convergencia de la norma del gradiente')
ax2.legend()
save_fig(fig, 'convergencia.png')
plt.close(fig)

Guardado -> figures\convergencia.png


## 9) Análisis de 400 Puntos Iniciales
Evaluación sistemática de ambos métodos desde 400 puntos distribuidos en el plano.

In [9]:
print("Iniciando análisis de 400 puntos iniciales...")

# Generar 400 puntos desde (-200,-200) hasta (200,200) en pasos de 20
x_range = np.arange(-200, 201, 20)
y_range = np.arange(-200, 201, 20)
puntos_400 = []
for x in x_range:
    for y in y_range:
        puntos_400.append(np.array([x, y], dtype=float))

# Almacenar resultados
resultados_md = []
resultados_newton = []

for i, punto in enumerate(puntos_400):
    if i % 50 == 0:
        print(f"Procesando punto {i+1}/400...")
    
    # Máximo Descenso con tolerancia más realista
    try:
        traj_md = maximo_descenso(punto, max_iter=500, tol=1e-4)
        f_final_md = f(traj_md[-1])
        grad_final_md = np.linalg.norm(grad_f(traj_md[-1]))
        iter_md = len(traj_md) - 1
        resultados_md.append({
            'punto_inicial': punto,
            'punto_final': traj_md[-1],
            'f_final': f_final_md,
            'grad_final': grad_final_md,
            'iteraciones': iter_md,
            'exito': grad_final_md < 1e-3  # Criterio más realista
        })
    except:
        resultados_md.append({
            'punto_inicial': punto,
            'punto_final': punto,
            'f_final': f(punto),
            'grad_final': np.linalg.norm(grad_f(punto)),
            'iteraciones': 0,
            'exito': False
        })
    
    # Método de Newton
    try:
        traj_newton = metodo_newton(punto, max_iter=100, tol=1e-6)
        f_final_newton = f(traj_newton[-1])
        grad_final_newton = np.linalg.norm(grad_f(traj_newton[-1]))
        iter_newton = len(traj_newton) - 1
        resultados_newton.append({
            'punto_inicial': punto,
            'punto_final': traj_newton[-1],
            'f_final': f_final_newton,
            'grad_final': grad_final_newton,
            'iteraciones': iter_newton,
            'exito': grad_final_newton < 1e-6
        })
    except:
        resultados_newton.append({
            'punto_inicial': punto,
            'punto_final': punto,
            'f_final': f(punto),
            'grad_final': np.linalg.norm(grad_f(punto)),
            'iteraciones': 0,
            'exito': False
        })

print("Análisis de 400 puntos completado.")

Iniciando análisis de 400 puntos iniciales...
Procesando punto 1/400...
Procesando punto 51/400...
Procesando punto 101/400...
Procesando punto 151/400...
Procesando punto 201/400...
Procesando punto 251/400...
Procesando punto 301/400...
Procesando punto 351/400...
Procesando punto 401/400...
Análisis de 400 puntos completado.


## 10) Visualización de los 400 Puntos

In [10]:
# Extraer datos para plotting
puntos_iniciales = np.array([r['punto_inicial'] for r in resultados_md])
f_finales_md = np.array([r['f_final'] for r in resultados_md])
f_finales_newton = np.array([r['f_final'] for r in resultados_newton])
iter_md = np.array([r['iteraciones'] for r in resultados_md])
iter_newton = np.array([r['iteraciones'] for r in resultados_newton])
exitos_md = np.array([r['exito'] for r in resultados_md])
exitos_newton = np.array([r['exito'] for r in resultados_newton])

# Crear figura con 6 subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Puntos iniciales
axes[0,0].scatter(puntos_iniciales[:, 0], puntos_iniciales[:, 1], 
                 c='blue', alpha=0.6, s=20)
axes[0,0].set_title('Distribución de 400 Puntos Iniciales')
axes[0,0].set_xlabel('x')
axes[0,0].set_ylabel('y')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].set_aspect('equal')

# 2. Valores finales de la función - Máximo Descenso
sc1 = axes[0,1].scatter(puntos_iniciales[:, 0], puntos_iniciales[:, 1], 
                       c=f_finales_md, 
                       cmap='RdBu_r', s=20, vmin=-1, vmax=1)
axes[0,1].set_title('Valor Final de f(x) - Máximo Descenso')
axes[0,1].set_xlabel('x')
axes[0,1].set_ylabel('y')
axes[0,1].grid(True, alpha=0.3)
axes[0,1].set_aspect('equal')
plt.colorbar(sc1, ax=axes[0,1])

# 3. Valores finales de la función - Newton
sc2 = axes[0,2].scatter(puntos_iniciales[:, 0], puntos_iniciales[:, 1], 
                       c=f_finales_newton, 
                       cmap='RdBu_r', s=20, vmin=-1, vmax=1)
axes[0,2].set_title('Valor Final de f(x) - Newton')
axes[0,2].set_xlabel('x')
axes[0,2].set_ylabel('y')
axes[0,2].grid(True, alpha=0.3)
axes[0,2].set_aspect('equal')
plt.colorbar(sc2, ax=axes[0,2])

# 4. Número de iteraciones - Máximo Descenso
sc3 = axes[1,0].scatter(puntos_iniciales[:, 0], puntos_iniciales[:, 1], 
                       c=iter_md, cmap='plasma', s=20)
axes[1,0].set_title('Iteraciones Requeridas - Máximo Descenso')
axes[1,0].set_xlabel('x')
axes[1,0].set_ylabel('y')
axes[1,0].grid(True, alpha=0.3)
axes[1,0].set_aspect('equal')
plt.colorbar(sc3, ax=axes[1,0])

# 5. Número de iteraciones - Newton
sc4 = axes[1,1].scatter(puntos_iniciales[:, 0], puntos_iniciales[:, 1], 
                       c=iter_newton, cmap='plasma', s=20)
axes[1,1].set_title('Iteraciones Requeridas - Newton')
axes[1,1].set_xlabel('x')
axes[1,1].set_ylabel('y')
axes[1,1].grid(True, alpha=0.3)
axes[1,1].set_aspect('equal')
plt.colorbar(sc4, ax=axes[1,1])

# 6. Comparación de éxito/fracaso
colores_md = ['red' if not ex else 'green' for ex in exitos_md]
colores_newton = ['red' if not ex else 'green' for ex in exitos_newton]
axes[1,2].scatter(puntos_iniciales[:, 0], puntos_iniciales[:, 1], 
                 c=colores_md, alpha=0.6, s=20, label='Máx. Descenso')
axes[1,2].scatter(puntos_iniciales[:, 0], puntos_iniciales[:, 1], 
                 c=colores_newton, alpha=0.3, s=10, label='Newton')
axes[1,2].set_title('Convergencia - Rojo: Fracaso, Verde: Éxito')
axes[1,2].set_xlabel('x')
axes[1,2].set_ylabel('y')
axes[1,2].grid(True, alpha=0.3)
axes[1,2].set_aspect('equal')
axes[1,2].legend()

plt.tight_layout()
save_fig(fig, 'analisis_400_puntos.png')
plt.close(fig)

Guardado -> figures\analisis_400_puntos.png


## 11) Estadísticas de los 400 Puntos

In [11]:
print("\n=== ANÁLISIS ESTADÍSTICO DE 400 PUNTOS ===")

# Estadísticas de éxito
exitos_md_count = np.sum(exitos_md)
exitos_newton_count = np.sum(exitos_newton)

print(f"Éxitos Máximo Descenso: {exitos_md_count}/400 ({exitos_md_count/400*100:.1f}%)")
print(f"Éxitos Newton: {exitos_newton_count}/400 ({exitos_newton_count/400*100:.1f}%)")

# Estadísticas de iteraciones (solo para casos exitosos)
iter_md_exitos = iter_md[exitos_md]
iter_newton_exitos = iter_newton[exitos_newton]

if len(iter_md_exitos) > 0:
    print(f"\nIteraciones Máximo Descenso (éxitos):")
    print(f"  Media: {np.mean(iter_md_exitos):.1f}")
    print(f"  Mediana: {np.median(iter_md_exitos):.1f}")
    print(f"  Mín: {np.min(iter_md_exitos)}")
    print(f"  Máx: {np.max(iter_md_exitos)}")

if len(iter_newton_exitos) > 0:
    print(f"\nIteraciones Newton (éxitos):")
    print(f"  Media: {np.mean(iter_newton_exitos):.1f}")
    print(f"  Mediana: {np.median(iter_newton_exitos):.1f}")
    print(f"  Mín: {np.min(iter_newton_exitos)}")
    print(f"  Máx: {np.max(iter_newton_exitos)}")

# Estadísticas de valores finales
print(f"\nValor final de f(x) - Máximo Descenso:")
print(f"  Media: {np.mean(f_finales_md):.2f}")
print(f"  Mín: {np.min(f_finales_md):.2f}")
print(f"  Máx: {np.max(f_finales_md):.2f}")

print(f"\nValor final de f(x) - Newton:")
print(f"  Media: {np.mean(f_finales_newton):.2f}")
print(f"  Mín: {np.min(f_finales_newton):.2f}")
print(f"  Máx: {np.max(f_finales_newton):.2f}")


=== ANÁLISIS ESTADÍSTICO DE 400 PUNTOS ===
Éxitos Máximo Descenso: 9/400 (2.2%)
Éxitos Newton: 247/400 (61.8%)

Iteraciones Máximo Descenso (éxitos):
  Media: 444.4
  Mediana: 500.0
  Mín: 0
  Máx: 500

Iteraciones Newton (éxitos):
  Media: 4.3
  Mediana: 4.0
  Mín: 0
  Máx: 12

Valor final de f(x) - Máximo Descenso:
  Media: 0.42
  Mín: -1.00
  Máx: 1.00

Valor final de f(x) - Newton:
  Media: -0.31
  Mín: -1.00
  Máx: 1.00


## 12) Gráficos Estadísticos Adicionales

In [12]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histograma de iteraciones
if len(iter_md_exitos) > 0 and len(iter_newton_exitos) > 0:
    axes[0,0].hist(iter_md_exitos, bins=30, alpha=0.7, label='Máx. Descenso', color='blue')
    axes[0,0].hist(iter_newton_exitos, bins=30, alpha=0.7, label='Newton', color='red')
    axes[0,0].set_xlabel('Iteraciones')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].set_title('Distribución de Iteraciones (Casos Exitosos)')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

# Boxplot de iteraciones
if len(iter_md_exitos) > 0 and len(iter_newton_exitos) > 0:
    datos_iter = [iter_md_exitos, iter_newton_exitos]
    axes[0,1].boxplot(datos_iter, tick_labels=['Máx. Descenso', 'Newton'])
    axes[0,1].set_ylabel('Iteraciones')
    axes[0,1].set_title('Distribución de Iteraciones')
    axes[0,1].grid(True, alpha=0.3)

# Histograma de valores finales
axes[1,0].hist(f_finales_md, bins=30, alpha=0.7, 
               label='Máx. Descenso', color='blue')
axes[1,0].hist(f_finales_newton, bins=30, alpha=0.7, 
               label='Newton', color='red')
axes[1,0].set_xlabel('f(x)')
axes[1,0].set_ylabel('Frecuencia')
axes[1,0].set_title('Distribución de Valores Finales')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Gráfico de convergencia por cuadrante
def clasificar_cuadrante(punto):
    x, y = punto
    if x >= 0 and y >= 0: return 'I'
    elif x < 0 and y >= 0: return 'II'
    elif x < 0 and y < 0: return 'III'
    else: return 'IV'

cuadrantes = [clasificar_cuadrante(p) for p in puntos_iniciales]
cuadrantes_unique = ['I', 'II', 'III', 'IV']
exitos_por_cuadrante_md = []
exitos_por_cuadrante_newton = []

for c in cuadrantes_unique:
    mask = np.array(cuadrantes) == c
    exitos_por_cuadrante_md.append(np.sum(exitos_md[mask]))
    exitos_por_cuadrante_newton.append(np.sum(exitos_newton[mask]))

x_pos = np.arange(len(cuadrantes_unique))
axes[1,1].bar(x_pos - 0.2, exitos_por_cuadrante_md, 0.4, label='Máx. Descenso')
axes[1,1].bar(x_pos + 0.2, exitos_por_cuadrante_newton, 0.4, label='Newton')
axes[1,1].set_xlabel('Cuadrante')
axes[1,1].set_ylabel('Éxitos')
axes[1,1].set_title('Éxitos por Cuadrante')
axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels(cuadrantes_unique)
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
save_fig(fig, 'estadisticas_400_puntos.png')
plt.close(fig)

print("\nGráficos del análisis de 400 puntos guardados.")

Guardado -> figures\estadisticas_400_puntos.png

Gráficos del análisis de 400 puntos guardados.
